In [164]:
from flickrapi import FlickrAPI
import pandas as pd

In [150]:
key = '0e4e7a8ceb3f350abd0a743812365c51'
secret = '3172a3f6ec67c50f'
sizes = ['url_c', 'url_z', 'url_m', 'url_n', 'url_s', 'url_t']

In [169]:
def get_photos(image_tag):
    extras = ','.join(sizes)
    flickr = FlickrAPI(key, secret)
    photos = flickr.walk(text=image_tag,  # it will search by image title and image tags
                            extras=extras,  # get the urls for each size we want
                            privacy_filter=1,  # search only for public photos
                            per_page=50,
                            sort='relevance')  # we want what we are looking for to appear first
    return photos

In [198]:
get_photos('patatasbravas')

<generator object FlickrAPI.data_walker at 0x7f9623599e58>

In [171]:
def get_url(photo):
    for i in range(len(sizes)):  # makes sure the loop is done in the order we want
        url = photo.get(sizes[i])
        #print('Encontrado el tamaño ',sizes[i])
        if url:  # if url is None try with the next size
            print('Encontrado el tamaño ',sizes[i])
            return url

In [172]:
def get_urls(image_tag, max):
    
    photos = get_photos(image_tag)
    counter=0
    urls=[]

    for photo in photos:
        if counter < max:
            counter += 1
            print("Fetching url for image number {}".format(counter))
            try:
                url = get_url(photo)  # get preffered size url
                urls.append(url)
            except:    
                 print("Url for image number {} could not be fetched".format(count))
            # if no url for the desired sizes then try with the next photo
        else:
            print("Done fetching urls, fetched {} urls out of {}".format(len(urls), max))
            break
    print("Writing out the urls in the current directory")
    urls = pd.Series(urls)
    urls.to_csv(image_tag + '_urls.csv')
    print('Listo!')
    

    return urls

In [199]:
urls = get_urls('patatasbravas', 1000)

Fetching url for image number 1
Encontrado el tamaño  url_c
Fetching url for image number 2
Encontrado el tamaño  url_c
Fetching url for image number 3
Encontrado el tamaño  url_c
Fetching url for image number 4
Encontrado el tamaño  url_c
Fetching url for image number 5
Encontrado el tamaño  url_c
Fetching url for image number 6
Encontrado el tamaño  url_c
Fetching url for image number 7
Encontrado el tamaño  url_c
Fetching url for image number 8
Encontrado el tamaño  url_c
Fetching url for image number 9
Encontrado el tamaño  url_c
Fetching url for image number 10
Encontrado el tamaño  url_c
Fetching url for image number 11
Encontrado el tamaño  url_c
Fetching url for image number 12
Encontrado el tamaño  url_c
Fetching url for image number 13
Encontrado el tamaño  url_c
Fetching url for image number 14
Encontrado el tamaño  url_c
Fetching url for image number 15
Encontrado el tamaño  url_c
Fetching url for image number 16
Encontrado el tamaño  url_c
Fetching url for image number 17


Fetching url for image number 151
Encontrado el tamaño  url_c
Fetching url for image number 152
Encontrado el tamaño  url_c
Fetching url for image number 153
Encontrado el tamaño  url_c
Fetching url for image number 154
Encontrado el tamaño  url_c
Fetching url for image number 155
Encontrado el tamaño  url_c
Fetching url for image number 156
Encontrado el tamaño  url_c
Fetching url for image number 157
Encontrado el tamaño  url_c
Fetching url for image number 158
Encontrado el tamaño  url_c
Fetching url for image number 159
Encontrado el tamaño  url_c
Fetching url for image number 160
Encontrado el tamaño  url_c
Fetching url for image number 161
Encontrado el tamaño  url_c
Fetching url for image number 162
Encontrado el tamaño  url_c
Fetching url for image number 163
Encontrado el tamaño  url_c
Fetching url for image number 164
Encontrado el tamaño  url_c
Fetching url for image number 165
Encontrado el tamaño  url_c
Fetching url for image number 166
Encontrado el tamaño  url_c
Fetching

Fetching url for image number 301
Encontrado el tamaño  url_c
Fetching url for image number 302
Encontrado el tamaño  url_c
Fetching url for image number 303
Encontrado el tamaño  url_c
Fetching url for image number 304
Encontrado el tamaño  url_c
Fetching url for image number 305
Encontrado el tamaño  url_c
Fetching url for image number 306
Encontrado el tamaño  url_c
Fetching url for image number 307
Encontrado el tamaño  url_c
Fetching url for image number 308
Encontrado el tamaño  url_c
Fetching url for image number 309
Encontrado el tamaño  url_c
Fetching url for image number 310
Encontrado el tamaño  url_c
Fetching url for image number 311
Encontrado el tamaño  url_c
Fetching url for image number 312
Encontrado el tamaño  url_z
Fetching url for image number 313
Encontrado el tamaño  url_c
Fetching url for image number 314
Encontrado el tamaño  url_c
Fetching url for image number 315
Encontrado el tamaño  url_c
Fetching url for image number 316
Encontrado el tamaño  url_c
Fetching

Fetching url for image number 451
Encontrado el tamaño  url_c
Fetching url for image number 452
Encontrado el tamaño  url_c
Fetching url for image number 453
Encontrado el tamaño  url_c
Fetching url for image number 454
Encontrado el tamaño  url_c
Fetching url for image number 455
Encontrado el tamaño  url_c
Fetching url for image number 456
Encontrado el tamaño  url_c
Fetching url for image number 457
Encontrado el tamaño  url_c
Fetching url for image number 458
Encontrado el tamaño  url_c
Fetching url for image number 459
Encontrado el tamaño  url_c
Fetching url for image number 460
Encontrado el tamaño  url_c
Fetching url for image number 461
Encontrado el tamaño  url_c
Fetching url for image number 462
Encontrado el tamaño  url_c
Fetching url for image number 463
Encontrado el tamaño  url_c
Fetching url for image number 464
Encontrado el tamaño  url_c
Fetching url for image number 465
Encontrado el tamaño  url_c
Fetching url for image number 466
Encontrado el tamaño  url_c
Fetching

Fetching url for image number 601
Encontrado el tamaño  url_c
Fetching url for image number 602
Encontrado el tamaño  url_c
Fetching url for image number 603
Encontrado el tamaño  url_c
Fetching url for image number 604
Encontrado el tamaño  url_c
Fetching url for image number 605
Encontrado el tamaño  url_c
Fetching url for image number 606
Encontrado el tamaño  url_c
Fetching url for image number 607
Encontrado el tamaño  url_c
Fetching url for image number 608
Encontrado el tamaño  url_c
Fetching url for image number 609
Encontrado el tamaño  url_c
Fetching url for image number 610
Encontrado el tamaño  url_c
Fetching url for image number 611
Encontrado el tamaño  url_c
Fetching url for image number 612
Encontrado el tamaño  url_c
Fetching url for image number 613
Encontrado el tamaño  url_c
Fetching url for image number 614
Encontrado el tamaño  url_m
Fetching url for image number 615
Encontrado el tamaño  url_c
Fetching url for image number 616
Encontrado el tamaño  url_c
Fetching

Fetching url for image number 751
Encontrado el tamaño  url_c
Fetching url for image number 752
Encontrado el tamaño  url_c
Fetching url for image number 753
Encontrado el tamaño  url_c
Fetching url for image number 754
Encontrado el tamaño  url_c
Fetching url for image number 755
Encontrado el tamaño  url_c
Fetching url for image number 756
Encontrado el tamaño  url_c
Fetching url for image number 757
Encontrado el tamaño  url_c
Fetching url for image number 758
Encontrado el tamaño  url_z
Fetching url for image number 759
Encontrado el tamaño  url_c
Fetching url for image number 760
Encontrado el tamaño  url_c
Fetching url for image number 761
Encontrado el tamaño  url_c
Fetching url for image number 762
Encontrado el tamaño  url_c
Fetching url for image number 763
Encontrado el tamaño  url_c
Fetching url for image number 764
Encontrado el tamaño  url_c
Fetching url for image number 765
Encontrado el tamaño  url_c
Fetching url for image number 766
Encontrado el tamaño  url_c
Fetching

Fetching url for image number 901
Encontrado el tamaño  url_c
Fetching url for image number 902
Encontrado el tamaño  url_c
Fetching url for image number 903
Encontrado el tamaño  url_z
Fetching url for image number 904
Encontrado el tamaño  url_z
Writing out the urls in the current directory
Listo!


In [191]:
urls.head()

0    https://live.staticflickr.com/2718/4247367545_...
1    https://live.staticflickr.com/3037/2813361589_...
2    https://live.staticflickr.com/6152/6225202837_...
3    https://live.staticflickr.com/5645/21302127116...
4    https://live.staticflickr.com/5647/22428081765...
dtype: object

In [200]:
!ls -l

total 776
-rw-rw-r-- 1 dsc dsc  54346 feb 24 22:56  cats_and_dogs.ipynb
-rw-rw-r-- 1 dsc dsc  28300 mar  6 23:24 'causa limeña_urls.csv'
-rw-rw-r-- 1 dsc dsc  61537 mar  6 23:43  chaufa_urls.csv
-rw-rw-r-- 1 dsc dsc   1302 mar  6 18:23  croquetas.csv
-rw-rw-r-- 1 dsc dsc  67405 mar  6 21:58  croquetas_urls.csv
-rw-rw-r-- 1 dsc dsc  66807 mar  6 22:16  fabada_urls.csv
-rw-rw-r-- 1 dsc dsc 145408 mar  7 00:25  flickr_scrap.ipynb
-rw-rw-r-- 1 dsc dsc  66907 mar  7 00:13  gazpacho_urls.csv
-rw-rw-r-- 1 dsc dsc  67093 mar  6 22:27  huancaina_urls.csv
-rw-rw-r-- 1 dsc dsc  66735 mar  6 22:06  paella_urls.csv
-rw-rw-r-- 1 dsc dsc  60636 mar  7 00:25  patatasbravas_urls.csv
-rw-rw-r-- 1 dsc dsc     22 feb 21 19:48  README.md
-rw-rw-r-- 1 dsc dsc   7413 mar  6 23:36  tacutacu_urls.csv
-rw-rw-r-- 1 dsc dsc  67058 mar  6 21:27 'tortilla de patata_urls.csv'


In [166]:
import requests
import os

def download_images(file_name, image_tag):
    urls = []
    doc = pd.read_csv(file_name, header=None)
    
    for row in doc.iterrows():
        if row[1][1].startswith('https'):
            print('yuhuuuu')
            urls.append(row[1][1])
    
    if not os.path.isdir(os.path.join('/home/dsc/tfm_data/',image_tag.split("_")[0])):
        os.mkdir(os.path.join('/home/dsc/tfm_data/', image_tag.split("_")[0]))
    
    
    for url in enumerate(urls):
        resp=requests.get(url[1], stream=True)
        path_to_write = os.path.join('/home/dsc/tfm_data/',image_tag.split("_")[0],url[1].split("/")[-1])
        outfile=open(path_to_write,'wb')
        outfile.write(resp.content)
        outfile.close()
        print("Done downloading {} of {}".format(url[0]+1,len(urls)))

In [201]:
download_images('patatasbravas_urls.csv', 'patatasbravas')

yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu
yuhuuuu


Done downloading 37 of 904
Done downloading 38 of 904
Done downloading 39 of 904
Done downloading 40 of 904
Done downloading 41 of 904
Done downloading 42 of 904
Done downloading 43 of 904
Done downloading 44 of 904
Done downloading 45 of 904
Done downloading 46 of 904
Done downloading 47 of 904
Done downloading 48 of 904
Done downloading 49 of 904
Done downloading 50 of 904
Done downloading 51 of 904
Done downloading 52 of 904
Done downloading 53 of 904
Done downloading 54 of 904
Done downloading 55 of 904
Done downloading 56 of 904
Done downloading 57 of 904
Done downloading 58 of 904
Done downloading 59 of 904
Done downloading 60 of 904
Done downloading 61 of 904
Done downloading 62 of 904
Done downloading 63 of 904
Done downloading 64 of 904
Done downloading 65 of 904
Done downloading 66 of 904
Done downloading 67 of 904
Done downloading 68 of 904
Done downloading 69 of 904
Done downloading 70 of 904
Done downloading 71 of 904
Done downloading 72 of 904
Done downloading 73 of 904
D

Done downloading 333 of 904
Done downloading 334 of 904
Done downloading 335 of 904
Done downloading 336 of 904
Done downloading 337 of 904
Done downloading 338 of 904
Done downloading 339 of 904
Done downloading 340 of 904
Done downloading 341 of 904
Done downloading 342 of 904
Done downloading 343 of 904
Done downloading 344 of 904
Done downloading 345 of 904
Done downloading 346 of 904
Done downloading 347 of 904
Done downloading 348 of 904
Done downloading 349 of 904
Done downloading 350 of 904
Done downloading 351 of 904
Done downloading 352 of 904
Done downloading 353 of 904
Done downloading 354 of 904
Done downloading 355 of 904
Done downloading 356 of 904
Done downloading 357 of 904
Done downloading 358 of 904
Done downloading 359 of 904
Done downloading 360 of 904
Done downloading 361 of 904
Done downloading 362 of 904
Done downloading 363 of 904
Done downloading 364 of 904
Done downloading 365 of 904
Done downloading 366 of 904
Done downloading 367 of 904
Done downloading 368

Done downloading 626 of 904
Done downloading 627 of 904
Done downloading 628 of 904
Done downloading 629 of 904
Done downloading 630 of 904
Done downloading 631 of 904
Done downloading 632 of 904
Done downloading 633 of 904
Done downloading 634 of 904
Done downloading 635 of 904
Done downloading 636 of 904
Done downloading 637 of 904
Done downloading 638 of 904
Done downloading 639 of 904
Done downloading 640 of 904
Done downloading 641 of 904
Done downloading 642 of 904
Done downloading 643 of 904
Done downloading 644 of 904
Done downloading 645 of 904
Done downloading 646 of 904
Done downloading 647 of 904
Done downloading 648 of 904
Done downloading 649 of 904
Done downloading 650 of 904
Done downloading 651 of 904
Done downloading 652 of 904
Done downloading 653 of 904
Done downloading 654 of 904
Done downloading 655 of 904
Done downloading 656 of 904
Done downloading 657 of 904
Done downloading 658 of 904
Done downloading 659 of 904
Done downloading 660 of 904
Done downloading 661

In [30]:
import csv